<a href="https://colab.research.google.com/github/myPar/NSU_Practice/blob/dev/baseline/RNNbaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Prepare

In [62]:
import pandas as pd
import numpy as np
from google.colab import drive
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
file_path = Path('/content/drive/MyDrive/Colab Notebooks/data.txt')
df = pd.read_csv(filepath_or_buffer=file_path, sep=' ', header=0)

df.head

<bound method NDFrame.head of              Date      Time  qo_lc[m3/d]  qw_lc[m3/d]  qg_lc[m3/d]  \
0     07-May-2014  10:13:26      109.280    -0.312890       3810.3   
1     07-May-2014  10:14:26      113.460    -0.437350       3774.0   
2     07-May-2014  10:15:26      131.630    -0.533550       3892.5   
3     07-May-2014  10:16:26      140.670     0.160910       4043.1   
4     07-May-2014  10:17:26      150.020    -1.145600       4134.6   
...           ...       ...          ...          ...          ...   
1264  08-May-2014  07:34:35       98.744     0.310070       3174.5   
1265  08-May-2014  07:35:35       72.045     0.162760       3134.1   
1266  08-May-2014  07:36:35       87.370    -0.125770       3340.4   
1267  08-May-2014  07:37:35       32.121     0.021522       2834.1   
1268  08-May-2014  07:38:35       35.914     0.228020       2612.4   

      qo_sc[Sm3/d]  qw_sc[Sm3/d]  qg_sc[Sm3/d]  qo_scnp[Sm3/d]  \
0          108.430     -0.314870       92700.0         108.430 

In [64]:
print(df.columns)

Index(['Date', 'Time', 'qo_lc[m3/d]', 'qw_lc[m3/d]', 'qg_lc[m3/d]',
       'qo_sc[Sm3/d]', 'qw_sc[Sm3/d]', 'qg_sc[Sm3/d]', 'qo_scnp[Sm3/d]',
       'qw_scnp[Sm3/d]', 'qg_scnp[Sm3/d]', 'Fo_lc[%]', 'Fw_lc[%]', 'Fg_lc[%]',
       'WLR[%]', 'GVF[%]', 'GLR[m3/m3]', 'BSW[%]', 'GOR[Sm3/Sm3]',
       'GOR1[Sm3/Sm3]', 'mo_lc[kg/d]', 'mw_lc[kg/d]', 'mg_lc[kg/d]',
       'm_lc[kg/d]', 'mo_sc[kg/d]', 'mw_sc[kg/d]', 'mg_sc[kg/d]',
       'Mu_o_lc[cP]', 'Mu_l_lc[cP]', 'Do_lc[g/cm3]', 'Dw_lc[g/cm3]',
       'Dg_lc[g/cm3]', 'Dm_lc[g/cm3]', 'Dl_lc[g/cm3]', 'bo[Sm3/m3]',
       'bw[Sm3/m3]', 'bg[Sm3/m3]', 'Z', 'Rst[Sm3/Sm3]', 'Rwst[Sm3/Sm3]',
       'rgmp[Sm3/Sm3]', 'N32[cps]', 'N81[cps]', 'N356[cps]', 'NTotal[cps]',
       'DeadTime[s]', 'SampleTime[s]', 'DPV[mbar]', 'PL[bara]', 'TL[DegC]',
       'TAMB[DegC]', 'Dyn_DP[mbar]', 'OperatingPointLE[1/m]',
       'OperatingPointHE[1/m]', 'OilPointLE[1/m]', 'OilPointHE[1/m]',
       'WaterPointLE[1/m]', 'WaterPointHE[1/m]', 'GasPointLE[1/m]',
       'GasPoin

In [65]:
selected_rows = ['DPV[mbar]', 'PL[bara]', 'qg_sc[Sm3/d]', 'qo_sc[Sm3/d]', 'TL[DegC]']
outputs = ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']
inputs = np.setdiff1d(selected_rows, outputs)

df = df[selected_rows]

print(df.head)
print("inputs: " + str(inputs))
print("outputs: " + str(outputs))

<bound method NDFrame.head of       DPV[mbar]  PL[bara]  qg_sc[Sm3/d]  qo_sc[Sm3/d]  TL[DegC]
0       1448.70    21.481       92700.0       108.430   -2.5452
1       1462.10    21.473       91895.0       112.580   -2.5323
2       1592.30    21.672       95860.0       130.550   -2.3931
3       1703.30    21.926      100690.0       139.440   -2.0451
4       1821.70    22.196      104290.0       148.640   -1.8009
...         ...       ...           ...           ...       ...
1264    1147.10    23.997       87402.0        97.703   -3.3542
1265    1090.50    23.919       85594.0        71.288   -3.2890
1266    1279.10    24.477       93660.0        86.386   -3.2191
1267     976.62    23.808       76564.0        31.789   -3.2819
1268        NaN       NaN       69609.0        35.567       NaN

[1269 rows x 5 columns]>
inputs: ['DPV[mbar]' 'PL[bara]' 'TL[DegC]']
outputs: ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']


In [66]:
# check categorical attributes
cardinality_hold = 0.15
expected_col_size = 1269

categorical_attributes = []

def check_attributes_info():
  for column in df.columns:
    col_data = df[column].to_numpy()
    size = len(col_data)
    
    assert size == expected_col_size
    cleared_data = col_data[~np.isnan(col_data)]
    
    card = len(np.unique(cleared_data))  # cardinality without Nan items
    skip_count = size - len(cleared_data)

    print(column + ": card=" + str(card) + "; skip: percent=" + str(skip_count * 100 / size) + ", count=" + str(skip_count))
    
    if card / size < cardinality_hold:
      categorical_attributes.append(card)

check_attributes_info()

if len(categorical_attributes) == 0:
  print("no categorical attributes")
else:
  print(categorical_attributes)

DPV[mbar]: card=1163; skip: percent=0.07880220646178093, count=1
PL[bara]: card=1042; skip: percent=0.07880220646178093, count=1
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
TL[DegC]: card=1214; skip: percent=0.07880220646178093, count=1
no categorical attributes


In [67]:
# supplement data
for column in df.columns:
  data =  np.array(df[column].to_numpy(), dtype=np.float64)

  for i in range(len(data)):
    if str(data[i]) == 'nan':
      data[i] = np.mean(data[~np.isnan(data)])
  df.drop([column], axis=1, inplace=True)
  df.insert(0, column, data)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [68]:
check_attributes_info()

TL[DegC]: card=1215; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
PL[bara]: card=1043; skip: percent=0.0, count=0
DPV[mbar]: card=1164; skip: percent=0.0, count=0


## Create model

In [69]:
# model parameters
batch_size = 60
timesteps = 60
epoch_count = 10
input_size = len(inputs)
output_size = len(outputs)
lr = 0.001

mse = tf.keras.losses.MeanSquaredError()

model = tf.keras.Sequential()
model.add(Input((timesteps, input_size)))
model.add(SimpleRNN(units=120, activation='tanh', return_sequences=True))
model.add(SimpleRNN(units=120, activation='tanh', return_sequences=True))
model.add(SimpleRNN(units=2, activation='tanh', return_sequences=True))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_12 (SimpleRNN)   (None, 60, 120)           14880     
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 60, 120)           28920     
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 60, 2)             246       
                                                                 
Total params: 44,046
Trainable params: 44,046
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(loss=mse, metrics=[mse], optimizer=Adam(learning_rate=lr))

## Prepare data

In [71]:
# normilize data ('min/max' normalization):
interval_th = [-1, 1]     # normalization interval for 'th' activation function
interval_sigmoid = [0, 1] # normalization interval for 'sigmoid' activation function

result_interval = interval_th
normalized_df = df = ((df - df.min()) / (df.max() - df.min())) * (result_interval[1] - result_interval[0]) + result_interval[0]

train_df, test_df = train_test_split(normalized_df, test_size=0.3)
print(train_df.head)
print(test_df.head)

train_input_data = train_df[inputs].to_numpy()
train_output_data = train_df[outputs].to_numpy()

test_input_data = test_df[inputs].to_numpy()
test_output_data = test_df[outputs].to_numpy()

print("data shapes:")
print(train_input_data.shape)
print(test_input_data.shape)

print(train_output_data.shape)
print(test_output_data.shape)

<bound method NDFrame.head of       TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]  PL[bara]  DPV[mbar]
62   -0.053767     -0.638801     -0.534039 -0.537590  -0.666367
752   0.236860     -0.056094      0.224093  0.191545  -0.079113
338   0.421386     -0.654443     -0.573553 -0.650033  -0.635346
959  -0.198703     -0.202721      0.107312  0.316191  -0.268193
557   0.761481      0.105644      0.327822 -0.171933   0.180431
...        ...           ...           ...       ...        ...
72   -0.102858     -0.637691     -0.551574 -0.464371  -0.698764
1071 -0.593470     -0.284689     -0.205317  0.385051  -0.517000
915  -0.059121     -0.394703      0.042256  0.364567  -0.343187
1084 -0.642108     -0.382506     -0.215045  0.385487  -0.573539
87   -0.173441      0.226456      0.340153  0.169754   0.185464

[888 rows x 5 columns]>
<bound method NDFrame.head of      TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]  PL[bara]  DPV[mbar]
53  -0.040646     -0.432715     -0.280141 -0.224232  -0.498209
842  0.246512     -0.

In [72]:
# formatting the data
def get_sequence(data):
  return np.array([data[i*timesteps : i*timesteps + timesteps] for i in range(data.shape[0] // timesteps)])

input_train_sequences = get_sequence(train_input_data)
output_train_sequences = get_sequence(train_output_data)

"""
input_train_sequences = list(input_train_sequences)
output_train_sequences = list(output_train_sequences)
"""

input_train_remaining = []
output_train_remaining = []

sequences_count = len(input_train_sequences)

assert sequences_count == len(output_train_sequences)

# add the remaining data:
if sequences_count * timesteps < train_input_data.shape[0]:
  input_train_remaining = np.array(train_input_data[sequences_count * timesteps:])

if sequences_count * timesteps < train_output_data.shape[0]:
  output_train_remaining = np.array(train_output_data[sequences_count * timesteps:])

print(len(input_train_sequences))
print(len(output_train_sequences))

for i in range(len(input_train_sequences)):
  print(str(i) + ": " + str(input_train_sequences[i].shape))

print("-----------")

for i in range(len(output_train_sequences)):
  print(str(i) + ": " + str(output_train_sequences[i].shape))

print("remaining data: ")
print(input_train_remaining.shape)
print(output_train_remaining.shape)

14
14
0: (60, 3)
1: (60, 3)
2: (60, 3)
3: (60, 3)
4: (60, 3)
5: (60, 3)
6: (60, 3)
7: (60, 3)
8: (60, 3)
9: (60, 3)
10: (60, 3)
11: (60, 3)
12: (60, 3)
13: (60, 3)
-----------
0: (60, 2)
1: (60, 2)
2: (60, 2)
3: (60, 2)
4: (60, 2)
5: (60, 2)
6: (60, 2)
7: (60, 2)
8: (60, 2)
9: (60, 2)
10: (60, 2)
11: (60, 2)
12: (60, 2)
13: (60, 2)
remaining data: 
(48, 3)
(48, 2)


In [73]:
batch_count = len(input_train_sequences)

input_train_tensor = input_train_sequences.reshape((batch_count, timesteps, input_size))
output_train_tensor = output_train_sequences.reshape((batch_count, timesteps, output_size))
"""
for i in range(epoch_count):
  for batch_idx in range(len(input_train_sequences)):
    input_train_sequence = input_train_sequences[batch_idx].reshape(1,timesteps, input_size)
    output_train_sequnce = output_train_sequences[batch_idx].reshape(1, timesteps, output_size)
    loss = model.train_on_batch(input_train_sequence, output_train_sequnce)

    print("epoch: " + str(i) + "; batch: " + str(batch_idx) + "; loss - " + str(loss))
"""
print(input_train_tensor.shape)
print(output_train_tensor.shape)

(14, 60, 3)
(14, 60, 2)


In [74]:
model.fit(input_train_tensor, output_train_tensor, batch_size=1, epochs=epoch_count)

Epoch 1/10
14/14 [==============================] - 2s 18ms/step - loss: 0.3023 - mean_squared_error: 0.3023
Epoch 2/10
14/14 [==============================] - 0s 26ms/step - loss: 0.1907 - mean_squared_error: 0.1907
Epoch 3/10
14/14 [==============================] - 0s 32ms/step - loss: 0.2070 - mean_squared_error: 0.2070
Epoch 4/10
14/14 [==============================] - 0s 32ms/step - loss: 0.1158 - mean_squared_error: 0.1158
Epoch 5/10
14/14 [==============================] - 0s 33ms/step - loss: 0.0694 - mean_squared_error: 0.0694
Epoch 6/10
14/14 [==============================] - 1s 39ms/step - loss: 0.0412 - mean_squared_error: 0.0412
Epoch 7/10
14/14 [==============================] - 0s 34ms/step - loss: 0.0524 - mean_squared_error: 0.0524
Epoch 8/10
14/14 [==============================] - 0s 34ms/step - loss: 0.0328 - mean_squared_error: 0.0328
Epoch 9/10
14/14 [==============================] - 0s 34ms/step - loss: 0.0270 - mean_squared_error: 0.0270
Epoch 10/10
14/14 [